**BERT 모델 불러오기**

In [ ]:
from transformers import BertModel, BertTokenizer
import torch

# 1. BERT 토크나이저와 모델을 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# tokenizer의 내장함수를 이용해 특수 토큰을 불러오는 함수를 작성해주세요
tokenizer.special_tokens_map


{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

**1.BERT Tokenizer의 특수 토큰**

아래 코드에서 특수 토큰을 불러오는 함수를 작성하고 각 토큰이 무엇을 의미하는지 적어주세요

- [UNK] : 토크나이저에 없는 단어들을 대체하는 토큰
- [SEP] : 문장을 구분하기 위한 토큰 및 문장의 마지막을 표시하는 토큰
- [PAD] :문장의 길이를 맞춰주기 위해 뒤에 추가하는 토큰(패딩)
- [CLS] : (처음)문장 시작 부호/(마지막)문맥에 대한 정보를 포함하여 분류에 사용하는 토큰
- [MASK] : 문장을 맞추기 위해 가리는 역할을 하는 토큰

**tokenizer를 이용해 문장을 토큰화**

토큰화를 통해 문자를 숫자로 변환하여 컴퓨터가 이해할 수 있도록 한다.

In [ ]:
# 입력 문장
sentence = "Hello, this is a sentence for tokenization."

# 문장 토큰화 및
inputs = tokenizer(sentence, return_tensors="pt") # 결과를 파이토치 텐서로 반환

# 토큰 ID를 다시 토큰으로 변환
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0].tolist())  # 텐서를 리스트로 변환

print("Token IDs:", inputs['input_ids'])
print("Tokens:", tokens)

Token IDs: tensor([[  101,  7592,  1010,  2023,  2003,  1037,  6251,  2005, 19204,  3989,
          1012,   102]])
Tokens: ['[CLS]', 'hello', ',', 'this', 'is', 'a', 'sentence', 'for', 'token', '##ization', '.', '[SEP]']


**2. 토큰화**
토큰화 이후에 [CLS] 토큰과 [SEP] 토큰이 추가된 이유를 작성하세요

- [CLS] 토큰이 추가된 이유 : [CLS]토큰은 입력 문장의 전체적인 정보를 포함하는 특징 벡터로 활용되기 위해 추가된다.
- [SEP] 토큰이 추가된 이유 : 문장의 끝을 나타내거나, 두 문장을 구분하는 역할을 한다. 현재는 문장이 하나이므로 문장의 끝을 나타낸다.

**감성 분류를 위한 사전 학습된 BERT 모델을 이용해 감성분류하기**

In [ ]:
import torch
from transformers import BertForSequenceClassification
from torch.nn.functional import softmax

# 감성 분류를 위한 사전 학습된 모델
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = BertForSequenceClassification.from_pretrained(model_name)

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [ ]:
# sentence에 대해 BERT를 이용해 감성분류 하는 함수
def sentiment_analysis(sentence):

    ############### EDIT###############
    'sentiment analysis 구현'
    inputs = tokenizer(sentence, return_tensors="pt")

    model.eval()

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    probs = softmax(logits, dim=1)  # 확률로 변환
    probs_list = probs.squeeze().tolist()

    ###################################

    # 클래스 레이블 정의
    class_label = ['매우 부정적', '부정적', '중립적', '긍정적', '매우 긍정적']

    # 결과 출력 (소수점 2자리로 출력)
    for i, prob in enumerate(probs_list):
        print(f"{class_label[i]}일 확률: {prob:.2f}")

**3. BERT를 이용하여 각각 class에 맞는 sentence 작성해보기**

1. 매우 부정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
2. 부정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
3. 중립적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
4. 긍정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
5. 매우 긍정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요

In [ ]:
# 매우 부정적인 문장 작성
sentence = "Terrible service! I regret ever coming here."

# 감성분석 수행
sentiment_analysis(sentence)


매우 부정적일 확률: 0.42
부정적일 확률: 0.21
중립적일 확률: 0.18
긍정적일 확률: 0.10
매우 긍정적일 확률: 0.09


In [ ]:
# 부정적인 문장 작성
sentence = "It wasn't as good as I expected."

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.30
부정적일 확률: 0.33
중립적일 확률: 0.26
긍정적일 확률: 0.08
매우 긍정적일 확률: 0.03


In [ ]:
# 중립적 문장 작성
sentence = "It was average, neither good nor bad."

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.22
부정적일 확률: 0.28
중립적일 확률: 0.33
긍정적일 확률: 0.14
매우 긍정적일 확률: 0.04


In [ ]:
# 긍정적 문장 작성
sentence = "The product works well, and I’m happy with it."

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.33
부정적일 확률: 0.27
중립적일 확률: 0.24
긍정적일 확률: 0.09
매우 긍정적일 확률: 0.06


In [ ]:
# 매우 긍정적인 문장 작성
sentence = "Everything about this place seems perfect, but something feels wrong."

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.67
부정적일 확률: 0.18
중립적일 확률: 0.09
긍정적일 확률: 0.04
매우 긍정적일 확률: 0.03


**4. BERT의 감성 분류 기준에 대해 생각을 적어보기**


BERT가 어떤 요소에 따라 감성 분류 기준을 나누는지에 대해 적어보세요

내가 생각하는 분류 기준 :

매우 긍정적인 문장을 ,(반점)기준으로 앞에는 긍정적인 어휘만 적었는데도 but이라는 단어 뒤에 나오는 wrong 같은 부정적인 뉘앙스의 어휘를 더 많이 반영한 것 같다. bert는 맥락을 파악하는 모델이고, 수많은 학습 데이터로 학습할 때 보통 but, however 등 문장의 전체적인 분위기가 뒤집히는 문장이 많았을 것으로 예상된다.